In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/kaggle/input/traincsv/train.csv')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
sns.heatmap(df.isnull(), yticklabels=False,cbar=False)

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.drop(['id'],axis=1,inplace=True)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
columns=['Gender','family_history_with_overweight','FAVC','CAEC','SMOKE','SCC','CALC','MTRANS']

In [ ]:
def category_onehot_multcols(df, multcolumns):
    df_final = df.copy()  
    for field in multcolumns:
        df1 = pd.get_dummies(df[field], drop_first=True).astype(int)
        df_final.drop([field], axis=1, inplace=True)
        df_final = pd.concat([df_final, df1], axis=1)
    return df_final

In [ ]:
df_final=category_onehot_multcols(df, columns)


In [ ]:
df_final['NObeyesdad'].unique()

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df_final['NObeyesdad']=label_encoder.fit_transform(df_final['NObeyesdad'])
df_final['NObeyesdad'].unique()

In [ ]:
df_final.shape

In [ ]:
df_final = df_final.loc[:,~df_final.columns.duplicated()]

In [ ]:
df_final.shape

In [ ]:
df_final

In [ ]:
df_dash = df_final.copy()

In [ ]:
X = df_final.drop(['NObeyesdad'],axis=1)


In [ ]:
cols = ['Age','Height','Weight','FCVC','NCP','CH2O','FAF','TUE','Male','yes','Frequently','Sometimes','no','Bike','Motorbike','Public_Transportation','Walking']
Y = df_dash.drop(cols,axis=1)

In [ ]:
Y.head()

In [ ]:
X.head()

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=42)


In [ ]:
classifier = xgb.XGBClassifier()
## Hyper Parameter Optimization

n_estimators = [100, 200, 300, 400, 500, 600, 700, 800,900, 1000]
booster=['gbtree','gblinear']
learning_rate=[0.05,0.1,0.15,0.20]
base_score=[0.25,0.5,0.75,1]


# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'learning_rate': uniform(0.01,0.3),
    'booster':booster,
    'base_score':base_score
    }
# Set up the random search with 4-fold cross validation
random_cv = RandomizedSearchCV(estimator=classifier,
            param_distributions=hyperparameter_grid, 
            cv=5, n_iter=20,
            scoring = 'accuracy',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)




In [ ]:
random_cv.fit(X_train,Y_train)

In [ ]:
random_cv.best_params_

In [ ]:
best_model = xgb.XGBClassifier(base_score=0.75, booster='gbtree',learning_rate=0.039992474745400866, n_estimators=900)

In [ ]:
best_model.fit(X_train,Y_train)

In [ ]:

Y_pred = best_model.predict(X_test)

accuracy = accuracy_score(Y_test, Y_pred)

accuracy

**Now we will engineer the test data.**


In [ ]:
df2 = pd.read_csv('/kaggle/input/testcsv/test.csv')

In [ ]:
df2.head()

In [ ]:
df2.drop(['id'], axis=1, inplace=True)

In [ ]:
columns2=['Gender','family_history_with_overweight','FAVC','CAEC','SMOKE','SCC','CALC','MTRANS']
def category_onehot_multcols(df, multcolumns):
    df_final = df.copy()
    for field in multcolumns:
        df1 = pd.get_dummies(df[field], drop_first=True).astype(int)
        df_final.drop([field], axis=1, inplace=True)
        df_final = pd.concat([df_final, df1], axis=1)
    return df_final

In [ ]:
df2_final = category_onehot_multcols(df2, columns2)

In [ ]:
df2_final = df2_final.loc[:,~df2_final.columns.duplicated()]

In [ ]:
df2_final.head()

In [ ]:
df2_final.shape

In [ ]:
X2 = df2_final

In [ ]:
Y_pred2 = best_model.predict(X2)

In [ ]:
df3 = pd.read_csv('/kaggle/input/playground-series-s4e2/sample_submission.csv')
id = df3['id']

In [ ]:

predictions = label_encoder.inverse_transform(Y_pred2)

output = pd.DataFrame(predictions, columns=['NObeyesdad'])
output['id'] = id 
output = output[['id', 'NObeyesdad']] 

In [ ]:
output.head(10)

In [ ]:

output.to_csv('predictions.csv', index=False)